In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

train = pd.read_csv('task2_lemmas_train', sep=',')
test = pd.read_csv('task2_lemmas_test', sep=',')
submission_example = pd.read_csv('task2_lemmas_sample_submission', sep=',')

In [2]:
f = open('task2_lemmas_train')
lines = []
for line in f:
    lines.append(line[:-2].decode('utf-8'))
    
train_data = []
for line in lines[1:]:
    array = line.split(',')
    if array[1].find('-') != -1:
        print array[1]
    train_data.append([array[1], zip(map(lambda x: x[:-2], array[2:]), map(lambda x: x[-1], array[2:]))])

anti-malarici
co-finanzierò
co-producerà
post-tangentopoli
italo-americano
neo-centriste
co-producevate
anti-repressivi
tedesco-orientali
non-violenza
ispano-marocchino
anti-cancerose
re-inietta
anti-sindacale
co-fonderemmo
cino-sovietici
clerico-fascisti
co-producei
mini-riforme
co-fondan
co-presiedente
anti-noriega
co-presiedesser
heiderian-lepenisti
anglo-iraniano
co-presiederà
dopo-voto
super-magri
anti-economico
anti-realistiche
co-finanzia
anti-occhetto
clerical-calcistico
co-produceste
ispano-marocchina
anti-centrosinistra
super-moderati
re-iniettereste
week-end
anti-repressiva
anti-governative
spy-story
anti-sovietici
co-fonderan
co-presiedessero
anti-amnistia
anti-democratico
video-giochi
anti-pinochetisti
anti-ferruzzi
afro-asiatici
ex-comunista
nero-blu
anti-serbe
co-presiedè
anti-scarcerazioni
co-producesser
classico-accademici
co-presiedesti
re-inietterai
re-iniettate
anti-ribasso
anglo-irlandese
filo-castriste
anti-unitaria
co-presiederemo
anti-giapponesi
post-elettorali


In [3]:
f = open('task2_lemmas_test')
test_words = []
for line in f:
    test_words.append(line[:-2].decode('utf-8').split(',')[-1])
test_words = test_words[1:]

In [31]:
def shortest_of(strings):
    return min(strings, key=len)

def long_substr(strings):
    substr = ""
    if not strings:
        return substr
    reference = shortest_of(strings) #strings[0]
    length = len(reference)
    #find a suitable slice i:j
    for i in xrange(length):
        #only consider strings long at least len(substr) + 1
        for j in xrange(i + len(substr) + 1, length + 1):
            candidate = reference[i:j]  # ↓ is the slice recalculated every time?
            if all(candidate in text for text in strings):
                substr = candidate
    return substr

def prefix(word, root):
    for i in range(len(word)-len(root) + 1):
        if word[i:i+len(root)] == root:
            return word[:i]
        
def suffix(word, root):
    ret = prefix(word[::-1], root[::-1])
    return ret[::-1] if ret else []

In [32]:
prefixes, suffixes, roots = [], [], []
N_suff, A_suff, V_suff, C_suff = [], [], [], []
for elem in train_data:
    strings = [elem[0]] + map(lambda x: x[0], elem[1])
    root = long_substr(strings)
    if root != '' and len(root) > 1:
        if root[0] == '-':
            root = root[1:]
        roots.append(root)
        for x in strings:
            pref, suff = prefix(x, root), suffix(x, root)
            if pref:
                prefixes.append(pref)
            if suff:
                suffixes.append(suff)
    C_suff.append(suffix(elem[0], root))
    for pair in elem[1]:
        if pair[1] == 'V':
            V_suff.append(suffix(pair[0], root))
        if pair[1] == 'A':
            A_suff.append(suffix(pair[0], root))
        if pair[1] == 'N':
            N_suff.append(suffix(pair[0], root))    
        
        

prefixes = sc.unique(prefixes)
suffixes = sc.unique(suffixes)
#roots = sc.unique(roots)

A_suff = sc.unique(A_suff)
N_suff = sc.unique(N_suff)
V_suff = sc.unique(V_suff)


new_pref = []
for elem in prefixes:
    if elem[-1] != '-':
        new_pref.append(elem)
prefixes = new_pref

In [37]:
print len(prefixes), len(suffixes), len(roots), len(train_data)

41 502 118597 118640


In [35]:
print prefixes

[u'c', u'capi', u'capo', u'con', u'coss', u'cu', u'cuoc', u'de', u'e', u'es', u'ess', u'esse', u'fo', u'm', u'mala', u'male', u'mani', u'mano', u'mett', u'mis', u'mu', u'percu', u'ric', u'ricu', u'rie', u'ripercu', u'ris', u'riu', u'rom', u'rup', u's', u'sa', u'sarebb', u'scu', u'si', u'soprass', u'sv', u'svi', u'u', u'v', u'vi']


In [36]:
print suffixes

[u'-chiave' u'a' u'a-estate' u'a-gol' u'a-paese' u'adonna' u'aforte' u'ai'
 u'amo' u'an' u'anno' u'ano' u'are' u'arsi' u'ate' u'avamo' u'avate' u'bbe'
 u'bber' u'bbero' u'bbi' u'ca' u'can' u'cano' u'ccia' u'cciamo' u'ccian'
 u'cciano' u'cciate' u'ccio' u'ce' u'cei' u'cemmo' u'cendo' u'cente'
 u'centi' u'cerai' u'ceran' u'ceranno' u'cere' u'cerebbe' u'cerebber'
 u'cerebbero' u'cerei' u'ceremmo' u'ceremo' u'cereste' u'ceresti' u'cerete'
 u'ceron' u'cerono' u'cer\xe0' u'cer\xf2' u'ces' u'cesse' u'cesser'
 u'cessero' u'cessi' u'cessimo' u'ceste' u'cesti' u'cete' u'ceva' u'cevamo'
 u'cevan' u'cevano' u'cevate' u'cevi' u'cevo' u'ci' u'cia' u'ciamo' u'cian'
 u'ciano' u'ciate' u'cio' u'cion' u'ciono' u'ciuta' u'ciute' u'ciuti'
 u'ciuto' u'co' u'con' u'cono' u'c\xe8' u'de' u'der' u'dere' u'dero' u'di'
 u'e' u'e-estati' u'e-gol' u'e-spia' u'ece' u'ecer' u'ecero' u'eci'
 u'edere' u'edonne' u'eforti' u'ei' u'emmo' u'en' u'endo' u'enere' u'enire'
 u'entare' u'ente' u'enti' u'erai' u'eran' u'eranno'

In [38]:
for i in range(10, 20):
    print train_data[i][0], ':', roots[i], ':', suffix(train_data[i][0], roots[i])
    print '==', C_suff[i]

accudisser : accudi : sser
== sser
diromperanno : diromper : anno
== anno
intercollegai : intercollega : i
== i
integrando : integra : ndo
== ndo
sbramerebbero : sbram : erebbero
== erebbero
stravaccherebbero : stravacc : herebbero
== herebbero
oltrepassasser : oltrepassa : sser
== sser
cauzionavan : cauziona : van
== van
scarrozzata : scarrozza : ta
== ta
intossicassero : intossica : ssero
== ssero


In [40]:
def find_root(word, roots=roots):
    def check_if_root(root):
        if len(root) <= len(word):
            for i in range(len(word) - len(root)):
                if word[i:i+len(root)] == root:
                    return True
        return False
    w_roots = []
    for root in roots:
        if check_if_root(root):
            w_roots.append(root)
    if w_roots:
        return w_roots[np.argmax(map(len, w_roots))]
    return '-'

def extract_main_part(word):
    main_part = '-'
    suffix1 = '-'
    for i in range(1, len(word)):
        if word[-i:] in suffixes:
            main_part, suffix1 = word[:-i], word[-i:]
    return main_part, suffix1

In [43]:
X_words = [x[0] for x in train_data]
y_words = [x[1][0][0] for x in train_data]
y_class = [x[1][0][1] for x in train_data]
print X_words[:10], '\n', y_words[:10], '\n', y_class[:10]

[u'vergognerete', u'amnistiavate', u'menomazione', u'sfaldavamo', u'sfodererei', u'ascondesti', u'edifichereste', u'maschieran', u'transennasser', u'computando'] 
[u'vergognare', u'amnistiare', u'menomazione', u'sfaldare', u'sfoderare', u'ascondere', u'edificare', u'maschiare', u'transennare', u'computare'] 
[u'V', u'V', u'N', u'V', u'V', u'V', u'V', u'V', u'V', u'V']


In [44]:
print extract_main_part(X_words[0])

(u'vergogn', u'erete')


0.9626

In [91]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 7), lowercase = True)
X = cv.fit_transform(X_words)
print np.mean(cross_val_score(LogisticRegression(), X, y_class, verbose=2))

[CV]  ................................................................
[CV] ................................................. , total=  21.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.2s remaining:    0.0s


[CV] ................................................. , total=  21.1s
[CV]  ................................................................
[CV] ................................................. , total=  26.5s
0.962879318504


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min finished


In [85]:
y_suffix = []
for word in y_words:
    ret1, ret2 = extract_main_part(word)
    if ret2 == '-':
        y_suffix.append(u'')
    else:
        y_suffix.append(ret2)

In [92]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 7), lowercase = True)
X = cv.fit_transform(X_words)
print np.mean(cross_val_score(LogisticRegression(), X, y_suffix, verbose=2))

/home/pavel/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV]  ................................................................
[CV] ................................................. , total= 1.5min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV] ................................................. , total= 1.7min
[CV]  ................................................................
[CV] ................................................. , total= 1.8min
0.966166612605


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.9min finished


In [97]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 7), lowercase = True)
X = cv.fit_transform(X_words)
X_test, X_train = X[:X.shape[0]/4], X[X.shape[0]/4:]
y_classes_test, y_classes_train = y_class[:X.shape[0]/4], y_class[X.shape[0]/4:]
y_suffix_test, y_suffix_train = y_suffix[:X.shape[0]/4], y_suffix[X.shape[0]/4:]

clf = LogisticRegression()
clf.fit(X_train, y_classes_train)
y_classes_pred = clf.predict(X_test)
clf.fit(X_train, y_suffix_train)
y_suff_pred = clf.predict(X_test)
 
print accuracy_score(y_classes_test, y_classes_pred)
print accuracy_score(y_suffix_test, y_suff_pred)

0.965037086986
0.968577208361


In [103]:
X_words_test = X_words[:X.shape[0]/4]
y_words_test = y_words[:X.shape[0]/4]

In [111]:
pred_words = []
true_words = []
for i in range(len(y_classes_pred)):
    true_words.append(y_words_test[i] + '+' + y_classes_test[i])
    main_part, suff = extract_main_part(X_words_test[i])
    pred_words.append(main_part + y_suff_pred[i] + '+' + y_classes_pred[i])

In [112]:
print accuracy_score(true_words, pred_words)

0.362070128119


In [113]:
print true_words[:10]

[u'vergognare+V', u'amnistiare+V', u'menomazione+N', u'sfaldare+V', u'sfoderare+V', u'ascondere+V', u'edificare+V', u'maschiare+V', u'transennare+V', u'computare+V']
